In [60]:
import json
import pennylane as qml
import pennylane.numpy as np

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):
    
    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.
    
    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground 
        state of the H3+ molecule with the given bond length.
    """
    symbols = ["H", "H", "H"]
    geometry = np.array([0., 0., 0., bond_length, 0., 0., bond_length / 2., bond_length * np.sin(np.pi / 3), 0.], requires_grad=True)
    basis_set = "sto-3g"
    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=1, basis=basis_set)
    electrons = 2
    hf = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    
    dev = qml.device("lightning.qubit", wires=qubits)

    def circuit(params, wires):
        qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

    @qml.qnode(dev, interface="autograd")
    def cost_fn(params):
        circuit(params, wires=range(qubits))
        return qml.expval(hamiltonian)
    np.random.seed(0)
    opt = qml.GradientDescentOptimizer(stepsize=1.0)
    theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)

    max_iterations = 300
    conv_tol = 1e-06

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)
        energy = cost_fn(theta)
        conv = np.abs(energy - prev_energy)
        if n % 4 == 0:
            print(f"Step = {n}, Energy = {energy:.8f} Ha")
        if conv <= conv_tol:
            break
    return energy


In [61]:
h3_ground_energy(0.8)

Step = 0, Energy = 1.02364486 Ha
Step = 4, Energy = -0.34143230 Ha
Step = 8, Energy = -0.36915251 Ha
Step = 12, Energy = -0.37032539 Ha
Step = 16, Energy = -0.37074192 Ha
Step = 20, Energy = -0.37111863 Ha
Step = 24, Energy = -0.37149055 Ha
Step = 28, Energy = -0.37185743 Ha
Step = 32, Energy = -0.37221713 Ha
Step = 36, Energy = -0.37256754 Ha
Step = 40, Energy = -0.37290671 Ha
Step = 44, Energy = -0.37323289 Ha
Step = 48, Energy = -0.37354463 Ha
Step = 52, Energy = -0.37384073 Ha
Step = 56, Energy = -0.37412032 Ha
Step = 60, Energy = -0.37438282 Ha
Step = 64, Energy = -0.37462795 Ha
Step = 68, Energy = -0.37485571 Ha
Step = 72, Energy = -0.37506632 Ha
Step = 76, Energy = -0.37526022 Ha
Step = 80, Energy = -0.37543801 Ha
Step = 84, Energy = -0.37560043 Ha
Step = 88, Energy = -0.37574830 Ha
Step = 92, Energy = -0.37588252 Ha
Step = 96, Energy = -0.37600400 Ha
Step = 100, Energy = -0.37611369 Ha
Step = 104, Energy = -0.37621250 Ha
Step = 108, Energy = -0.37630134 Ha
Step = 112, Energy = 

array(-0.37700083)

In [12]:
bond_length = 1.5
symbols = ["H", "H", "H"]
geometry = np.array([[0., 0., 0.], [0., 0., bond_length], [0., bond_length * np.sin(np.pi / 3), bond_length / 2.]])
hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=1)

In [14]:
electrons = 3
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 1 0 0 0]


In [32]:
singles, doubles = qml.qchem.excitations(3, 6)

In [33]:
dev = qml.device("lightning.qubit", wires=qubits)

def circuit(params, wires):
    qml.AllSinglesDoubles(params, wires, hf, singles, doubles)

@qml.qnode(dev, interface="autograd")
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(hamiltonian)

In [34]:
opt = qml.GradientDescentOptimizer(stepsize=0.8)
np.random.seed(0)  # for reproducibility
theta = np.random.normal(0, np.pi, len(singles) + len(doubles), requires_grad=True)
print(theta)

[ 5.54193389  1.25713095  3.07479606  7.03997361  5.86710646 -3.07020901
  2.98479079 -0.47550269]


In [35]:
max_iterations = 300
conv_tol = 1e-06

for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy = cost_fn(theta)

    conv = np.abs(energy - prev_energy)

    if n % 4 == 0:
        print(f"Step = {n}, Energy = {energy:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy:.8f} Ha")
print("\n" f"Optimal value of the circuit parameters = {theta}")

Step = 0, Energy = -0.52471941 Ha
Step = 4, Energy = -1.04750849 Ha
Step = 8, Energy = -1.19995694 Ha
Step = 12, Energy = -1.21046175 Ha
Step = 16, Energy = -1.21163833 Ha
Step = 20, Energy = -1.21208234 Ha
Step = 24, Energy = -1.21240180 Ha
Step = 28, Energy = -1.21268257 Ha
Step = 32, Energy = -1.21294500 Ha
Step = 36, Energy = -1.21319512 Ha
Step = 40, Energy = -1.21343526 Ha
Step = 44, Energy = -1.21366675 Ha
Step = 48, Energy = -1.21389069 Ha
Step = 52, Energy = -1.21410812 Ha
Step = 56, Energy = -1.21432003 Ha
Step = 60, Energy = -1.21452740 Ha
Step = 64, Energy = -1.21473116 Ha
Step = 68, Energy = -1.21493219 Ha
Step = 72, Energy = -1.21513129 Ha
Step = 76, Energy = -1.21532922 Ha
Step = 80, Energy = -1.21552665 Ha
Step = 84, Energy = -1.21572419 Ha
Step = 88, Energy = -1.21592239 Ha
Step = 92, Energy = -1.21612171 Ha
Step = 96, Energy = -1.21632256 Ha

Final value of the ground-state energy = -1.21647440 Ha

Optimal value of the circuit parameters = [ 3.16654162  1.08837056  3.

In [62]:

# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"


# These are the public test cases
test_cases = [
    ('1.5', '-1.232574'),
    ('0.8', '-0.3770325')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.5'...
Step = 0, Energy = -0.14959469 Ha
Step = 4, Energy = -1.02822328 Ha
Step = 8, Energy = -1.21187469 Ha
Step = 12, Energy = -1.22226158 Ha
Step = 16, Energy = -1.22416780 Ha
Step = 20, Energy = -1.22516159 Ha
Step = 24, Energy = -1.22596821 Ha
Step = 28, Energy = -1.22670639 Ha
Step = 32, Energy = -1.22739329 Ha
Step = 36, Energy = -1.22802823 Ha
Step = 40, Energy = -1.22860877 Ha
Step = 44, Energy = -1.22913372 Ha
Step = 48, Energy = -1.22960354 Ha
Step = 52, Energy = -1.23002013 Ha
Step = 56, Energy = -1.23038648 Ha
Step = 60, Energy = -1.23070632 Ha
Step = 64, Energy = -1.23098380 Ha
Step = 68, Energy = -1.23122324 Ha
Step = 72, Energy = -1.23142891 Ha
Step = 76, Energy = -1.23160489 Ha
Step = 80, Energy = -1.23175498 Ha
Step = 84, Energy = -1.23188263 Ha
Step = 88, Energy = -1.23199095 Ha
Step = 92, Energy = -1.23208270 Ha
Step = 96, Energy = -1.23216029 Ha
Step = 100, Energy = -1.23222582 Ha
Step = 104, Energy = -1.23228111 Ha
Step = 108, Ener